In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, VBox, HBox, Output

# Constants
hbar = 1.055e-34  # Reduced Planck constant (Joule-seconds)
m_e = 9.11e-31  # Electron rest mass (kg)
eV_to_J = 1.602e-19  # 1 eV in Joules

# Effective mass options (in terms of electron mass m_e)
effective_masses = {
    "GaAs_conduction": 0.067 * m_e,
    "GaAs_valence_hh": 0.62 * m_e,  # Heavy hole
    "GaAs_valence_lh": 0.15 * m_e   # Light hole
}

# Initial layer configuration
initial_layers = [
    (100, "GaAs"),
    *[(90, "AlGaAs"), (10, "GaAs")] * 5,
    (300, "AlGaAs"), (200, "GaAs"), (300, "AlGaAs"), (100, "GaAs"),
    (300, "AlGaAs"), (70, "GaAs"), (300, "AlGaAs"), (50, "GaAs"),
    (300, "AlGaAs"), (30, "GaAs"), (300, "AlGaAs"), (20, "GaAs"),
    (300, "AlGaAs"), (50, "GaAs")
]

# Functions for calculations
def calculate_gaas_bandgap(T):
    return 1.57 - 0.057 * (1 + 2 / (np.exp(240 / T) - 1))

def conduction_band_offset(x):
    if x < 0.41:
        return 0.79 * x
    else:
        return 0.475 - 0.335 * x + 0.143 * (x ** 2)

def valence_band_offset(x):
    return -0.46 * x

def calculate_band_edges(material, gaas_bandgap, x_algaas):
    if material == "GaAs":
        upper = gaas_bandgap / 2
        lower = -gaas_bandgap / 2
    elif "Al" in material and "GaAs" in material:  # Handle AlGaAs
        upper = (gaas_bandgap / 2) + conduction_band_offset(x_algaas)
        lower = -(gaas_bandgap / 2) + valence_band_offset(x_algaas)
    else:
        raise ValueError("Unknown material: " + material)
    return upper, lower

def calculate_energy_levels(n_max, width, effective_mass):
    d = width * 1e-10  # Convert width from Å to meters
    levels = [(hbar ** 2 * (np.pi ** 2) * n ** 2) / (2 * effective_mass * d ** 2) / eV_to_J for n in range(1, n_max + 1)]
    return levels

def generate_potential_plot(T, x_algaas, n_max, manual_y_range, y_min, y_max, valence_mass_choice, layers):
    # Set fixed conduction mass and selected valence mass
    conduction_mass = effective_masses['GaAs_conduction']
    valence_mass = effective_masses[valence_mass_choice]

    # Calculate bandgap
    gaas_bandgap = calculate_gaas_bandgap(T)

    # Adjust layers if x_algaas == 0
    if x_algaas == 0:
        layers = [(int(gaas_bandgap * 100), "GaAs") for _, material in layers]

    # Initialize potential arrays
    x = []
    upper_potential = []
    lower_potential = []
    energy_levels = []

    # Generate potential profile
    current_x = 0
    n_colors = ["red", "blue", "green", "orange", "purple"]  # Colors for energy levels

    for width, material in layers:
        upper, lower = calculate_band_edges(material, gaas_bandgap, x_algaas)
        x_region = np.linspace(current_x, current_x + width, 100)
        upper_region = np.full_like(x_region, upper)
        lower_region = np.full_like(x_region, lower)
        x.extend(x_region)
        upper_potential.extend(upper_region)
        lower_potential.extend(lower_region)
        if material == "GaAs":
            conduction_levels = calculate_energy_levels(n_max, width, conduction_mass)
            valence_levels = calculate_energy_levels(n_max, width, valence_mass)
            energy_levels.append((current_x, conduction_levels, valence_levels))
        current_x += width

    # Plot the potential well
    fig, ax = plt.subplots(figsize=(20, 5))
    ax.plot(x, upper_potential, color='black', label='Upper Potential Limit')
    ax.plot(x, lower_potential, color='black', label='Lower Potential Limit')

    # Add energy levels to the plot
    for pos, conduction_levels, valence_levels in energy_levels:
        for i, (E_c, E_v) in enumerate(zip(conduction_levels, valence_levels), 1):
            ax.hlines(E_c, pos, pos + width, color=n_colors[(i - 1) % len(n_colors)], linestyle='--',
                      label=f'Conduction n={i}' if pos == 0 else "")
            ax.hlines(-E_v, pos, pos + width, color=n_colors[(i - 1) % len(n_colors)], linestyle='-')

    # Set y-axis range
    if manual_y_range:
        ax.set_ylim(y_min, y_max)
    else:
        ax.autoscale(enable=True, axis='y')

    # Add labels, grid, and legend
    ax.set_xlabel('Position (Å)', fontsize=14)
    ax.set_ylabel('Energy (eV)', fontsize=14)
    ax.set_title(f'Potential Well for GaAs and AlGaAs Layers at T={T} K')
    ax.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

# Interactive UI for Layers
layers_output = Output()
layers = initial_layers.copy()

def add_layer(material, thickness):
    layers.append((thickness, material))
    with layers_output:
        layers_output.clear_output()
        print("Layers:")
        for idx, (t, m) in enumerate(layers):
            print(f"{idx + 1}: {m}, {t} Å")

def reset_layers():
    global layers
    layers = initial_layers.copy()
    with layers_output:
        layers_output.clear_output()
        print("Layers reset to initial configuration.")

def remove_all_layers():
    global layers
    layers.clear()
    with layers_output:
        layers_output.clear_output()
        print("All layers removed.")

material_dropdown = widgets.Dropdown(options=["GaAs", "AlGaAs"], value="GaAs", description="Material")
thickness_input = widgets.IntText(value=100, description="Thickness (Å)")
add_layer_button = widgets.Button(description="Add Layer")
reset_button = widgets.Button(description="Reset Layers")
remove_all_button = widgets.Button(description="Remove All Layers")

add_layer_button.on_click(lambda b: add_layer(material_dropdown.value, thickness_input.value))
reset_button.on_click(lambda b: reset_layers())
remove_all_button.on_click(lambda b: remove_all_layers())

# UI Widgets
T_slider = widgets.FloatSlider(value=300, min=1, max=500, step=10, description='T (K)')
x_slider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.01, description='Al-Fraction (x)')
n_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description='Quantum Number')
manual_y_toggle = widgets.ToggleButton(value=True, description='Manual Y-Range?')
y_min_box = widgets.FloatText(value=-1.2, description='Y-Min (eV)')
y_max_box = widgets.FloatText(value=1.2, description='Y-Max (eV)')
valence_mass_dropdown = widgets.Dropdown(options=["GaAs_valence_hh", "GaAs_valence_lh"],
                                         value="GaAs_valence_hh", description='Valence Mass')

# Layout and Interaction
def update_plot(T, x_algaas, n_max, manual_y_range, y_min, y_max, valence_mass_choice):
    generate_potential_plot(T, x_algaas, n_max, manual_y_range, y_min, y_max, valence_mass_choice, layers)

plot_ui = VBox([
    HBox([T_slider, x_slider]),
    HBox([n_slider, manual_y_toggle, y_min_box, y_max_box]),
    valence_mass_dropdown,
    add_layer_button, reset_button, remove_all_button, material_dropdown, thickness_input, layers_output
])

out = widgets.interactive_output(update_plot, {
    'T': T_slider,
    'x_algaas': x_slider,
    'n_max': n_slider,
    'manual_y_range': manual_y_toggle,
    'y_min': y_min_box,
    'y_max': y_max_box,
    'valence_mass_choice': valence_mass_dropdown
})

# Display
print("Initial Layers Loaded.")
reset_layers()
display(plot_ui, out)
